Read two saved template Models 
Interpolate Ge and Ph data to create input and output targets for training.  

1. Load existing template, relational and generative NN models
2. Initialize coherence model
3. Train coherence model
4. Push Template date through Relational, Generative and Trained Coherence Models
5. Graph context values over morphology for different time values
6. Calculate coherence values for graphing over morphology
7. Graph coherence field for time values

In [1]:
import torch as torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import os
import numpy as np
import json 
import math as m
import random
import time
import NNArt as gy
import NNArt1 as gy1
from sympy.utilities.iterables import multiset_permutations

import plotly as py
import plotly.offline as offline
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.io as pio

CURRENT_MODEL_SAVE_DIR = './save/'

offline.init_notebook_mode(connected=True) 
np.set_printoptions(precision=3, suppress=True)


In [2]:
class Net(nn.Module):
    def __init__(self, layerSizes, name):
        super(Net, self).__init__()
        self.shape = layerSizes
        self.name = name
        self.seq = nn.Sequential(
            nn.Linear(layerSizes[0],layerSizes[1]),
            nn.ELU(),          
            nn.Linear(layerSizes[1],layerSizes[2]),
            nn.ELU(),            
            nn.Linear(layerSizes[2],layerSizes[3]),
            nn.ELU()
        )          
    def forward(self, x):
        return self.seq(x) 

# class NetEEL(nn.Module):
#     def __init__(self, layerSizes, name):
#         super(NetEEL, self).__init__()
#         self.shape = layerSizes
#         self.name = name
#         self.seq = nn.Sequential(
#             nn.Linear(layerSizes[0],layerSizes[1]),
#             nn.ELU(),          
#             nn.Linear(layerSizes[1],layerSizes[2]),
#             nn.ELU(),            
#             nn.Linear(layerSizes[2],layerSizes[3]),
#             nn.ELU(),            

#         )          
#     def forward(self, x):
#         return self.seq(x) 

In [3]:
'''
1. Load existing template, relational and generative NN models


'''
tmModel0 = Net([2,10,15,21],'tmModel0')
tmModel1 = Net([2,10,15,21],'tmModel1')
relModel = Net([21,15,10,3],'relModel')
genModel = Net([3,10,15,21],'genModel')

PATH=CURRENT_MODEL_SAVE_DIR+'templateModel0.pth'
tmModel0.load_state_dict(torch.load(PATH))
PATH=CURRENT_MODEL_SAVE_DIR+'templateModel1.pth'
tmModel1.load_state_dict(torch.load(PATH))
PATH=CURRENT_MODEL_SAVE_DIR+'REL PhRc-T0T1-mask 11p05.pth'
relModel.load_state_dict(torch.load(PATH))
PATH=CURRENT_MODEL_SAVE_DIR+'GEN RcGen-T0T1-mask 11p05.pth'
genModel.load_state_dict(torch.load(PATH))
tmModel0.eval()
tmModel1.eval()
relModel.eval()
genModel.eval()

tmModel0.shape, tmModel1.shape,genModel.shape

([2, 10, 15, 21], [2, 10, 15, 21], [3, 10, 15, 21])

In [5]:
def createRcCohTargets(tmModels, relModel, numSamples=1,rmRange = 1.):
    #generate random templateModel input
    ge = np.random.random([numSamples,2])*rmRange
    relIn = torch.Tensor(ge)
    tmOut0 = tmModels[0](torch.Tensor(relIn))
    tmOut1 = tmModels[1](torch.Tensor(relIn))
    
    #get relational Model outputs for template inputs
    relCout0 = relModel(tmOut0)
    relCout1 = relModel(tmOut1)  
    rc0 = relCout0.data.numpy()
    rc1 = relCout1.data.numpy()
#     print('{}\n{}\n'.format(rc0,rc1))

    rcList = []
    cohList = []   
#     for i in range(rc0.shape[0]):
    for i in range(numSamples):
        delta = 2*(np.random.random()-.5)
#         print('delta: ',delta)
        #clone before changing
        rcc = np.array(rc0[i])
        rcc[2] = delta #set new context value
        rcList.append(rcc)

        c0 = rc0[i,2]
        c1 = rc1[i,2]
        mse0 = (c0-delta)**2
        mse1 = (c1-delta)**2
        if c0-delta < 0:
            mse0 = -mse0
        if c1-delta < 0:
            mse1 = -mse1
            
#         print('c0,c1,delta:\n {}\n{}\n{}\n'.format(c0,c1,delta))

        cohList.append([mse0,mse1])
        
    return torch.Tensor(rcList), torch.Tensor(cohList)

torch.set_printoptions(precision=20)
r,c = createRcCohTargets([tmModel0, tmModel1], relModel, numSamples=2)
print('{}\n{}'.format(np.array(r),np.array(c)))


[[ 0.352  0.845  0.007]
 [ 0.682  0.631 -0.518]]
[[0.003 0.   ]
 [0.245 0.17 ]]


In [8]:
def trainRcCoh(tmModels, relModel, cohModel, parms):
    epochs = parms['epochs']
    lRate = parms['lRate']
    numSamples = parms['numSamples']
  
    minLoss = 10
    maxLoss = -10
    criterion = torch.nn.MSELoss(reduction='sum')
    optimizer = torch.optim.Adam(cohModel.parameters(), lr=lRate)
    for epoch in range(epochs):
        cohIn, cohTarget = createRcCohTargets(tmModels,relModel,numSamples,rmRange=1.)
        optimizer.zero_grad()
        cohOut = cohModel(cohIn)
        loss = criterion(cohOut, cohTarget)
        (loss).backward()
        optimizer.step()
        minLoss = min(loss.data.item(), minLoss)
        maxLoss = max(loss.data.item(), maxLoss)
            
        if (epoch)%(epochs/10) == 0:
            print("Epoch {0:,.0f} - loss: {1:,.5f}".format(epoch, loss.data.item()))
    print("Epoch {0:,.0f} - loss: {1:,.5f}:{2:,.5f}".format(epoch+1, minLoss, maxLoss))
 
    return

In [13]:
'''
2. Initialize coherence model

'''

#initialize network
manualSeed = 1
np.random.seed(manualSeed)
torch.manual_seed(manualSeed)

cohModel = NetEEL([3,5,3,2],'cohModel')
cohModel.train()

# device = torch.device("cuda:0")
device = torch.device("cpu")
if device:
    tmModel0.to(device)
    tmModel1.to(device)
    relModel.to(device)
    cohModel.to(device)
    print("Transfer to use ", device)

tmModel0.shape,relModel.shape, cohModel.shape

Transfer to use  cpu


([2, 10, 15, 21], [21, 15, 10, 3], [3, 5, 3, 2])

In [18]:
''' 
3. Train coherence model

'''
start = time.time()

parms={ 'epochs':100000,'lRate':.0001,'numSamples':1,'device':device}
trainRcCoh([tmModel0,tmModel1],relModel,cohModel,parms)

print('time: ', time.time()-start)
cohModel.train = False

Epoch 0 - loss: 0.00082
Epoch 10,000 - loss: 0.00080
Epoch 20,000 - loss: 0.00010
Epoch 30,000 - loss: 0.00038
Epoch 40,000 - loss: 0.00098
Epoch 50,000 - loss: 0.00027
Epoch 60,000 - loss: 0.00200
Epoch 70,000 - loss: 0.00039
Epoch 80,000 - loss: 0.00003
Epoch 90,000 - loss: 0.00056
Epoch 100,000 - loss: 0.00000:0.43873
time:  109.1226737499237


In [ ]:
'''
parms={ 'epochs':100000,'lRate':.01,'numSamples':1,'device':device}
Epoch 100,000 - loss: 0.00000:3.36957
parms={ 'epochs':100000,'lRate':.005,'numSamples':1,'device':device}
Epoch 100,000 - loss: 0.00000:0.43895
parms={ 'epochs':100000,'lRate':.001,'numSamples':1,'device':device}
Epoch 100,000 - loss: 0.00000:0.43133
parms={ 'epochs':100000,'lRate':.0005,'numSamples':1,'device':device}
Epoch 100,000 - loss: 0.00000:0.41156
parms={ 'epochs':100000,'lRate':.0001,'numSamples':1,'device':device}
Epoch 100,000 - loss: 0.00000:0.43873

'''
pass

In [19]:
'''
4. Push Template date through Relational, Generative and Trained Coherence Models

'''
morphIndices=[0.,0.1,.2,.3,.5,.7,.9,1.0, 1.1]
timeIndices=[0.,0.1,.2,.3,.5,.7,.9,1.0, 1.1]

#sample source templates to get ph samples
relIn = gy.createRmInput(morphIndices, timeIndices, sPermutation=[])
tmOut0 = tmModel0(torch.Tensor(relIn))
tmOut1 = tmModel1(torch.Tensor(relIn))

#use template generated ph samples to feed through contex models
relCout0 = relModel(tmOut0)
relCout1 = relModel(tmOut1)

genOut0 = genModel(relCout0)
genOut1 = genModel(relCout1)

cohOut0 = cohModel(relCout0)
cohOut1 = cohModel(relCout1)

relIn.shape, tmOut0.shape, relCout0.shape, genOut0.shape, cohOut0.shape

((9, 9, 2),
 torch.Size([9, 9, 21]),
 torch.Size([9, 9, 3]),
 torch.Size([9, 9, 21]),
 torch.Size([9, 9, 2]))

In [20]:
'''
5. Graph context values over morphology for different time values

'''
#relCout0 for 0-8 time intervals
gTp0T0 = np.array([relIn[0,:,1],relCout0[0,:,2].detach().numpy()])
gTp0Tp2 = np.array([relIn[2,:,1],relCout0[2,:,2].detach().numpy()])
gTp0Tp5 = np.array([relIn[4,:,1],relCout0[4,:,2].detach().numpy()])
gTp0Tp9 = np.array([relIn[6,:,1],relCout0[6,:,2].detach().numpy()])
gTp0T1 = np.array([relIn[7,:,1],relCout0[7,:,2].detach().numpy()])
gTp0T1p1 = np.array([relIn[8,:,1],relCout0[8,:,2].detach().numpy()])

#relCout1 for 0-8 time intervals
gTp1T0 = np.array([relIn[0,:,1],relCout1[0,:,2].detach().numpy()])
gTp1Tp2 = np.array([relIn[2,:,1],relCout1[2,:,2].detach().numpy()])
gTp1Tp5 = np.array([relIn[4,:,1],relCout1[4,:,2].detach().numpy()])
gTp1Tp9 = np.array([relIn[6,:,1],relCout1[6,:,2].detach().numpy()])
gTp1T1 = np.array([relIn[7,:,1],relCout1[7,:,2].detach().numpy()])
gTp1T1p1 = np.array([relIn[8,:,1],relCout1[8,:,2].detach().numpy()])

gx = [gTp0T0, gTp1T0, gTp0Tp2, gTp1Tp2, gTp0Tp5, gTp1Tp5,
    gTp0Tp9, gTp1Tp9,gTp0T1p1,gTp1T1p1]


In [21]:
layout = go.Layout(
    title = 'Initial Curves',
    height = 700,
    width = 800,
    hovermode= 'closest',
    xaxis= dict(
        title= 'Morphology',
        range = [-.1,1.5],
    ),
    yaxis=dict(
        title= 'Context',
        range = [-.6,1.],
    )
)
layout.title = 'Context values over morphology for different time values' 
f = gy.graphCurves(gx, layout, labels=['gTp0T0','gTp1T0', 'gTp0Tp2', 'gTp1Tp2', 'gTp0T0p5', 'gTp1T0p5', 
                                       'gTp0T0p9','gTp1T0p9',  'gTp0T0_1p1','gTp1T0_1p1'], 
                   dashRange=gy.rangeList([range(2,4),range(6,8)]))

offline.iplot(f)


In [22]:
'''
6. Calculate coherence values for graphing over morphology

'''

t=8
#structure context values for graphing
con0 = np.array([relIn[t,:,1],relCout0[t,:,2].data.numpy()]) 
con1 = np.array([relIn[t,:,1],relCout1[t,:,2].data.numpy()])

#coherence values
#reshape into a 1,1,9 list of 9 coh values
# ch0 = np.reshape(cohOut0[t].data.numpy(), [1,2,-1])
# ch1 = np.reshape(cohOut1[t].data.numpy(), [1,2,-1])
ch0 = np.reshape(cohOut1[t].data.numpy()[:,0],[1,1,-1])
coh0 = np.array([relIn[t,:,1],ch0[0,0]])
cohD0 =  np.array(con0)
cohD0[1,:] = coh0[1,:]+con0[1,:]


In [23]:
gxx = [con0,con1,coh0,cohD0]
layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.4,1.2],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.4,.5],
    )
)
fxx = gy.graphCurves(gxx, layout, labels=['context0','context1','coherence0','coherence1', 'con0+coh0','con1+coh1'], dashRange=range(0,2))
offline.iplot(fxx)

In [24]:
'''
7. Graph coherence field for time values

'''

#create a matrix of mesh values
mm,cc = np.meshgrid(np.arange(-0., 1.1, .1), np.arange(-.5, .9, .1))

t = .9
ma = np.array(mm)
ca = np.array(cc)
xList = []
yList = []
uList = []
vList = []
for mi in range(ma.shape[0]):
    for ci in range(ca.shape[1]):
        cIn = torch.Tensor([[t,ma[mi,ci], ca[mi,ci]]])
        cohOut = cohModel(cIn)
        x = ma[mi,ci]
        y = ca[mi,ci]
        u = 0
        v = cohOut.data.numpy()[0,0] #extract value 0.0716
        xList.append(x)
        yList.append(y)
        uList.append(u)
        vList.append(v)

xx = np.array(xList)
yy = np.array(yList)
uu = np.array(uList)
vv = np.array(vList)

fig0 = ff.create_quiver(xx, yy, uu, vv,
       scale=.4,
       arrow_scale=.4,
       name='quiver'
       #,line=dict(width=2)
      )

fig0.layout.title = 'Coherence field for time ' + str(t)
fig0.add_trace(f.data[6]) 
fig0.add_trace(f.data[7]) 
offline.iplot(fig0)


In [25]:
#create a matrix of mesh values
mm,cc = np.meshgrid(np.arange(-0., 1.1, .1), np.arange(-.5, .9, .1))

t = .5
ma = np.array(mm)
ca = np.array(cc)
xList = []
yList = []
uList = []
vList = []
for mi in range(ma.shape[0]):
    for ci in range(ca.shape[1]):
#         print('({},{})'.format(mi,ci))
        cIn = torch.Tensor([[t,ma[mi,ci], ca[mi,ci]]])
        cohOut = cohModel(cIn)
        x = ma[mi,ci]
        y = ca[mi,ci]
        u = 0
        v = cohOut.data.numpy()[0,0] #extract value 0.0716
        xList.append(x)
        yList.append(y)
        uList.append(u)
        vList.append(v)
#         print('xy[{}, {}], uv[{}, {}]'.format(x,y,u,v))

xx = np.array(xList)
yy = np.array(yList)
uu = np.array(uList)
vv = np.array(vList)

fig0 = ff.create_quiver(xx, yy, uu, vv,
       scale=.4,
       arrow_scale=.4,
       name='quiver'
       #,line=dict(width=2)
      )
fig0.layout.title = 'Coherence field for time ' + str(t)
fig0.add_trace(f.data[4]) 
fig0.add_trace(f.data[5]) 
offline.iplot(fig0)


In [26]:
#create a matrix of mesh values
mm,cc = np.meshgrid(np.arange(-0., 1.2, .1), np.arange(-.5, 1, .1))

t = .2
ma = np.array(mm)
ca = np.array(cc)
xList = []
yList = []
uList = []
vList = []
for mi in range(ma.shape[0]):
    for ci in range(ca.shape[1]):
        cIn = torch.Tensor([[t,ma[mi,ci], ca[mi,ci]]])
        cohOut = cohModel(cIn)
        x = ma[mi,ci]
        y = ca[mi,ci]
        u = 0
        v = cohOut.data.numpy()[0,0] #extract value 0.0716
        xList.append(x)
        yList.append(y)
        uList.append(u)
        vList.append(v)


xx = np.array(xList)
yy = np.array(yList)
uu = np.array(uList)
vv = np.array(vList)

fig0 = ff.create_quiver(xx, yy, uu, vv,
       scale=.4,
       arrow_scale=.4,
       name='quiver'
       #,line=dict(width=2)
      )

fig0.layout.title = 'Coherence field for time ' + str(t)
fig0.add_trace(f.data[2]) 
fig0.add_trace(f.data[3]) 
offline.iplot(fig0)


In [27]:
#create a matrix of mesh values
mm,cc = np.meshgrid(np.arange(-0., 1.2, .1), np.arange(-.5, 1, .1))

t = 0
ma = np.array(mm)
ca = np.array(cc)
xList = []
yList = []
uList = []
vList = []
for mi in range(ma.shape[0]):
    for ci in range(ca.shape[1]):
        cIn = torch.Tensor([[t,ma[mi,ci], ca[mi,ci]]])
        cohOut = cohModel(cIn)
        x = ma[mi,ci]
        y = ca[mi,ci]
        u = 0
        v = cohOut.data.numpy()[0,0] #extract value 0.0716
        xList.append(x)
        yList.append(y)
        uList.append(u)
        vList.append(v)


xx = np.array(xList)
yy = np.array(yList)
uu = np.array(uList)
vv = np.array(vList)

fig0 = ff.create_quiver(xx, yy, uu, vv,
       scale=.4,
       arrow_scale=.4,
       name='quiver'
       #,line=dict(width=2)
      )

fig0.layout.title = 'Coherence field for time ' + str(t)
fig0.add_trace(f.data[0]) 
fig0.add_trace(f.data[1]) 
offline.iplot(fig0)


In [28]:
'''
7. Save two template interpolation model to file

'''

# PATH = CURRENT_MODEL_SAVE_DIR +'GEN PhRcPh-T0T1-mask 11p05.pth'
# torch.save(phModel.state_dict(), PATH)
# PATH = CURRENT_MODEL_SAVE_DIR +'REL PhRc-T0T1-mask 11p05.pth'
# torch.save(geModel.state_dict(), PATH)

PATH = CURRENT_MODEL_SAVE_DIR +'cohModel.pth'
torch.save(cohModel.state_dict(), PATH)
PATH

'./save/cohModel.pth'

In [ ]:
'''
9. Generate blender data from template interpolation model output

#phOut.shape torch.Size([10, 7, 23])

'''

# phOut = phModel(torch.Tensor(torch.Tensor(inpt0)))

# x = phOut.detach().numpy()
# fb = gy.FormatNetworkToBlender(x, xOffset=0, yOffset=7, zOffset=14, colPoints=7)

# with open(CURRENT_MODEL_SAVE_DIR +'btest.json', 'w') as fp:
#     json.dump(fb.tolist(),fp)
    
# x.shape, fb.shape